# **Preparação**




Instala o Wandb

In [ ]:
pip install wandb

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 163kB 17.1MB/s 
     |████████████████████████████████| 133kB 17.6MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=cd1b323b4d14308a6f2b8fa95c9b53bb05de186c768e0b148471319586c3b3e3
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=f82c721d2f6a45cbbc3f821912cbb3bec9fdf1d9b29463dd09aca35367c64d53
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


Realiza as importações principais

In [ ]:

import os
import cv2
import json
import h5py
import wandb
import random
import progressbar
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications import imagenet_utils
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2 
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet import MobileNet 
from keras.applications.densenet import DenseNet169 


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical


from imutils import paths
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import AveragePooling2D

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint
from wandb.keras import WandbCallback



Definindo os caminhos

In [ ]:
# define the path to the output training, validation, and testing
# HDF5 files
TRAIN_HDF5 = "raiox/hdf5/train.hdf5"
VAL_HDF5 = "raiox/hdf5/val.hdf5"
TEST_HDF5 = "raiox/hdf5/test.hdf5"


Montando o drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Pegando os dados do drive 

In [ ]:
# only in case load data from drive
!mkdir raiox
!cp -r /content/drive/MyDrive/Projeto3/hdf5 raiox/hdf5
!cp -r /content/drive/MyDrive/Projeto3/output raiox/output

Login no wandb

In [ ]:
API_KEY = "45ac723c1674f216bf7eb795bc3bc7cf591d5bd2"
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Configuração das variáveis globais

In [ ]:
#Colocar

BASE_MODELS = {
  "resnet" : 0,
  "irv2" : 1,
  "iv3" : 2,
  "vgg16" : 3,
  "vgg19" : 4,
  "mobile" : 5,
  "dense" : 6
}

BASE_SHAPES = {
    "irv2" : (299, 299, 3),
    "iv3" : (299, 299, 3),
    "resnet" : (224, 224, 3),
    "vgg16" : (224, 224, 3),
    "vgg19" : (224, 224, 3),
    "mobile" : (224, 224, 3),
    "dense" : (224, 224, 3)
}

OPTIMIZERS = {
    "Adam" : {
        "learning_rate" : 0.001,
        "beta_1" : 0.9
    },
    "SGD" : {
        "learning_rate" : 0.001,
        "momentum" : 0.0
    }
}



**Definição das classes de preprocessamento**:

SimplePreprocessor:

In [ ]:
class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

Image to Array:


In [ ]:
class ImageToArrayPreprocessor:
	def __init__(self, dataFormat=None):
		# store the image data format
		self.dataFormat = dataFormat

	def preprocess(self, image):
		# apply the Keras utility function that correctly rearranges
		# the dimensions of the image
		return img_to_array(image, data_format=self.dataFormat)

HDF5 dataset generators:

In [ ]:
#colocar
class HDF5DatasetGenerator:
	def __init__(self, dbPath, batchSize, preprocessors=None, aug=None, binarize=True, classes=2):
		# store the batch size, preprocessors, and data augmentor,
		# whether or not the labels should be binarized, along with
		# the total number of classes
		self.batchSize = batchSize
		self.preprocessors = preprocessors
		self.aug = aug
		self.binarize = binarize
		self.classes = classes

		# open the HDF5 database for reading and determine the total
		# number of entries in the database
		self.db = h5py.File(dbPath, "r")
		self.numImages = self.db["labels"].shape[0]

	def generator(self, passes=np.inf):
		# initialize the epoch count
		epochs = 0

		# keep looping infinitely -- the model will stop once we have
		# reach the desired number of epochs
		while epochs < passes:
			# loop over the HDF5 dataset
			for i in np.arange(0, self.numImages, self.batchSize):
				# extract the images and labels from the HDF dataset
				images = self.db["images"][i: i + self.batchSize]
				labels = self.db["labels"][i: i + self.batchSize]

				# check to see if the labels should be binarized
				if self.binarize:
					labels = to_categorical(labels,
						self.classes)

				# check to see if our preprocessors are not None
				if self.preprocessors is not None:
					# initialize the list of processed images
					procImages = []

					# loop over the images
					for image in images:
						# loop over the preprocessors and apply each
						# to the image
						for p in self.preprocessors:
							image = p.preprocess(image)

						# update the list of processed images
						procImages.append(image)

					# update the images array to be the processed
					# images
					images = np.array(procImages)

				# if the data augmenator exists, apply it
				if self.aug is not None:
					(images, labels) = next(self.aug.flow(images,
						labels, batch_size=self.batchSize))

				# yield a tuple of images and labels
				yield (images, labels)

			# increment the total number of epochs
			epochs += 1

	def close(self):
		# close the database
		self.db.close()

Função para criar arquitetura desejada com o camada fully connected

In [ ]:
#colocar
def load_base_model(base_model_name, fc_size, num_classes, dropout):
  """ Load pre-trained base network, add a global average pooling layer and two
  fully connected layers for fine-tuning, and freeze all base layers """

  base_model = BASE_MODELS[base_model_name]

  if (base_model == 0):
    base = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  elif (base_model == 1):
    base = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  elif (base_model == 2):
    base = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  elif (base_model == 3):
    base = VGG16(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  elif (base_model == 4):
    base = VGG19(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  elif (base_model == 5):
    base = MobileNet(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  elif (base_model == 6):
    base = DenseNet169(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))
  else:
     base = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(BASE_SHAPES[base_model_name]))

  head_model = base.output
  head_model = Flatten(name="flatten")(head_model)
  head_model = Dense(fc_size, activation="relu")(head_model)
  head_model = Dropout(dropout)(head_model)
  head_model = Dense(num_classes, activation="softmax")(head_model)

  model = Model(inputs=base.input, outputs=head_model)
  
  # freeze all base layers
  for layer in base.layers:
    layer.trainable = False

  # optimizer
  opt = RMSprop(lr=0.001)
  
  model.compile(optimizer=opt, loss= "categorical_crossentropy", metrics=['accuracy'])
  print("[INFO] compiling model...")


  return model

Pretreinamento com parametros congelados

In [ ]:
#colocar
def training_model(args):
  """ Load a pre-trained model and pre-train it for some epochs (args.pretrain_epochs).
  Then freeze learned layers up to args.freeze_layer, and continue training the remaining
  layers for the rest of the epochs (args.epochs) """

  #id = wandb.util.generate_id()

  #wandb.init(config = args, project= args["project_name"], name = args["run_name"], id=id, resume="allow", group = id)
  wandb.init(config = args, project= args["project_name"], name = args["run_name"], job_type="train")
  callbacks = [WandbCallback()]

  # construct the training image generator for data augmentation
  aug =  ImageDataGenerator(rotation_range=15, fill_mode="nearest")
 

  # initialize the image preprocessors
  #sp = SimplePreprocessor(args["img_dim"], args["img_dim"])
  iap = ImageToArrayPreprocessor()

  # initialize the training and validation dataset generators
  trainGen = HDF5DatasetGenerator(TRAIN_HDF5, args["batch_size"], aug= aug, preprocessors=[iap], classes= args["num_classes"])
  valGen = HDF5DatasetGenerator(VAL_HDF5, args["batch_size"], aug= aug, preprocessors=[iap], classes= args["num_classes"])
  testGen = HDF5DatasetGenerator(TEST_HDF5, 30, preprocessors=[iap], classes= args["num_classes"])

  # load the model with parameters freeze
  model = load_base_model(args["architecture"], args["fc_size"], args["num_classes"],  args["dropout"])
 
  history = model.fit(trainGen.generator(),
          steps_per_epoch=trainGen.numImages // args["batch_size"],
          validation_data=valGen.generator(),
          validation_steps=valGen.numImages // args["batch_size"],
          epochs=args["pretrain_epochs"],
          max_queue_size=10,
          callbacks = callbacks,
          verbose=1)

  # unfreezing the layer of base model
  for layer in model.layers[args["freeze_layers"]:]:
    layer.trainable = True

  #Optimizer
  if (args["optimizer"] == "SGD"):
    opt = SGD(learning_rate = OPTIMIZERS["SGD"]["learning_rate"], momentum = OPTIMIZERS["SGD"]["momentum"])
  else:
    opt = Adam(learning_rate = OPTIMIZERS["Adam"]["learning_rate"], beta_1 = OPTIMIZERS["Adam"]["beta_1"])


  print("[INFO] re-compiling model...")
  model.compile(loss=args["loss"], optimizer=opt, metrics=["accuracy"])

  # train the model again, this time fine-tuning *both* the final set
  # of CONV layers along with our set of FC layers
  print("[INFO] fine-tuning model...")
  # train the network
  history = model.fit(trainGen.generator(),
          steps_per_epoch=trainGen.numImages // args["batch_size"],
          validation_data=valGen.generator(),
          validation_steps=valGen.numImages // args["batch_size"],
          epochs= args["epochs"],
          max_queue_size=10,
          callbacks = callbacks,
          verbose=1)
  
  #log performance network on test data
  print("[INFO] evaluating network...")
  classNames = ["COVID", "Normal"]
  predictions = model.predict(testGen.generator(1) ,verbose=1)
  testY = np.array(testGen.db["labels"])
  pred = predictions.argmax(axis=1)

  #calculating metric
  f1 = f1_score(y_true= testY, y_pred = pred)
  rec = recall_score(y_true = testY, y_pred = pred)
  precision = precision_score(y_true = testY, y_pred = pred)
  
  #export metrics to wandb
  print("[INFO] saving logs on wandb...")
  wandb.log({'f1': f1})
  wandb.log({'recall': rec})
  wandb.log({'precision': precision})

  # precision- recall curve
  wandb.log({"pr" : wandb.plot.pr_curve(y_true = testY, y_probas = predictions, labels = classNames)})
  # confusion matrix
  wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None, preds=pred, y_true= testY , class_names= classNames)})
  
  # save the model to a file
  print("[INFO] serializing model in wandb...")

  name_model = "model_" + args["run_name"] + ".h5"
  wandb.save(name_model)
  
  # close the HDF5 datasets
  trainGen.close()
  valGen.close()

  return model

Função responsável por rodar o experimento

In [ ]:
MODELS_PARAMS = {
  "resnet" : 25636712,
  "irv2" : 55873736,
  "iv3" : 23851784,
  "vgg16" : 138357544,
  "vgg19" : 143667240,
  "mobile" : 4253864,
  "dense" : 14307880
}
default_test = {
    "fc_size"     : 256,
    "batch_size" : 128 ,
    "epochs" : 5,
    "pretrain_epochs" : 2,
    "optimizer" : "SGD",
    "learning_rate" : 0.001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,   
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss"     : "categorical_crossentropy"
}

args = {
    "project_name" : "xrays_covid19",
    "run_name" : "vgg19_v2", 
    "architecture" : "vgg19",
    "num_params" : 143667240,
    "freeze_layers" : 12, 
    "batch_size" : 128 ,
    "fc_size"     : 256,
    "pretrain_epochs" : 2,
    "epochs" : 5,
    "optimizer" : "SGD",
    "learning_rate" : 0.001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,  
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss" : "categorical_crossentropy"  
}


#modelo = training_model(args)


In [ ]:
version = "v4"

testes = [{
    "project_name" : "xrays_covid19_reborn",
    "run_name" : "vgg19_{}".format(version), 
    "architecture" : "vgg19",
    "num_params" : 143667240,
    "freeze_layers" : 7, 
    "batch_size" : 64 ,
    "fc_size"     : 256,
    "pretrain_epochs" : 2,
    "epochs" : 7,
    "optimizer" : "SGD",
    "learning_rate" : 0.001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,  
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss" : "categorical_crossentropy"  
  }, {
    "project_name" : "xrays_covid19_reborn",
    "run_name" : "vgg16_{}".format(version), 
    "architecture" : "vgg16",
    "num_params" : 138357544,
    "freeze_layers" : 7, 
    "batch_size" : 128 ,
    "fc_size"     : 256,
    "pretrain_epochs" : 2,
    "epochs" : 5,
    "optimizer" : "SGD",
    "learning_rate" : 0.0001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,  
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss" : "categorical_crossentropy"  
  }, {
    "project_name" : "xrays_covid19_reborn",
    "run_name" : "resnet_{}".format(version), 
    "architecture" : "resnet",
    "num_params" : 25636712,
    "freeze_layers" : 96, 
    "batch_size" : 64 ,
    "fc_size"     : 256,
    "pretrain_epochs" : 2,
    "epochs" : 5,
    "optimizer" : "SGD",
    "learning_rate" : 0.001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,  
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss" : "categorical_crossentropy"  
  }, {
    "project_name" : "xrays_covid19_reborn",
    "run_name" : "dense_{}".format(version), 
    "architecture" : "dense",
    "num_params" : 14307880,
    "freeze_layers" : 500, 
    "batch_size" : 64 ,
    "fc_size"     : 256,
    "pretrain_epochs" : 2,
    "epochs" : 5,
    "optimizer" : "Adam",
    "learning_rate" : 0.001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,  
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss" : "categorical_crossentropy"  
  }, {
    "project_name" : "xrays_covid19_reborn",
    "run_name" : "mobile_{}".format(version), 
    "architecture" : "mobile",
    "num_params" : 4253864,
    "freeze_layers" : 58, 
    "batch_size" : 64 ,
    "fc_size"     : 256,
    "pretrain_epochs" : 2,
    "epochs" : 5,
    "optimizer" : "Adam",
    "learning_rate" : 0.0001,
    "beta_1" : 0.9,
    "momentum" : 0.0,
    "dropout" : 0.5,  
    "img_dim" : 224,
    "num_classes" : 2,
    "n_train" :6480,
    "n_valid" : 720,
    "image_normalize" : False,
    "loss" : "categorical_crossentropy"  
  }
          
]


In [ ]:

#colocar
for teste in testes:
  modelo = training_model(teste)
  print("[INFO] finishing the train of model {}...".format(teste["run_name"]))
  wandb.run.finish()


[INFO] compiling model...
Epoch 1/2
90/90 [==============================] - 176s 2s/step - loss: 35.8106 - accuracy: 0.7685 - val_loss: 0.2004 - val_accuracy: 0.9134
Epoch 2/2
90/90 [==============================] - 175s 2s/step - loss: 0.6086 - accuracy: 0.8801 - val_loss: 0.2295 - val_accuracy: 0.9332
[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/7
90/90 [==============================] - 176s 2s/step - loss: 0.2471 - accuracy: 0.9161 - val_loss: 0.1594 - val_accuracy: 0.9489
Epoch 2/7
90/90 [==============================] - 174s 2s/step - loss: 0.1269 - accuracy: 0.9517 - val_loss: 0.1468 - val_accuracy: 0.9503
Epoch 3/7
90/90 [==============================] - 174s 2s/step - loss: 0.1069 - accuracy: 0.9628 - val_loss: 0.1224 - val_accuracy: 0.9616
Epoch 4/7
90/90 [==============================] - 174s 2s/step - loss: 0.0841 - accuracy: 0.9620 - val_loss: 0.1819 - val_accuracy: 0.9460
Epoch 5/7
90/90 [==============================] - 174s 2s/step - loss: 0.09

epoch,6
loss,0.06485
accuracy,0.97274
val_loss,0.09243
val_accuracy,0.97443
_runtime,1591
_timestamp,1619567951
_step,13
best_val_loss,0.08596
best_epoch,5
f1,0.96639


epoch,▁▂▁▂▃▅▆▇█
loss,█▁▁▁▁▁▁▁▁
accuracy,▁▄▆▇█████
val_loss,▇█▅▄▃▆▁▁▁
val_accuracy,▁▃▅▅▆▅▇██
_runtime,▁▂▃▄▄▅▆▇██████
_timestamp,▁▂▃▄▄▅▆▇██████
_step,▁▂▂▃▃▄▄▅▅▆▆▇▇█
f1,▁
recall,▁
precision,▁


[INFO] compiling model...
Epoch 1/2
45/45 [==============================] - 146s 3s/step - loss: 70.5143 - accuracy: 0.7141 - val_loss: 0.3670 - val_accuracy: 0.8047
Epoch 2/2
45/45 [==============================] - 144s 3s/step - loss: 0.9054 - accuracy: 0.8271 - val_loss: 0.2149 - val_accuracy: 0.9234
[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/5
45/45 [==============================] - 145s 3s/step - loss: 0.2533 - accuracy: 0.9236 - val_loss: 0.1516 - val_accuracy: 0.9453
Epoch 2/5
45/45 [==============================] - 144s 3s/step - loss: 0.2189 - accuracy: 0.9314 - val_loss: 0.1428 - val_accuracy: 0.9516
Epoch 3/5
45/45 [==============================] - 143s 3s/step - loss: 0.1871 - accuracy: 0.9436 - val_loss: 0.1686 - val_accuracy: 0.9438
Epoch 4/5
45/45 [==============================] - 145s 3s/step - loss: 0.1871 - accuracy: 0.9462 - val_loss: 0.1827 - val_accuracy: 0.9203
Epoch 5/5
45/45 [==============================] - 143s 3s/step - loss: 0.17

epoch,4
loss,0.17126
accuracy,0.94878
val_loss,0.13338
val_accuracy,0.95
_runtime,1029
_timestamp,1619568985
_step,11
best_val_loss,0.13338
best_epoch,4
f1,0.96533


...

epoch,4
loss,0.10688
accuracy,0.96128
val_loss,0.13103
val_accuracy,0.95028
_runtime,1283
_timestamp,1619571543
_step,11
best_val_loss,0.13103
best_epoch,4
f1,0.92765


epoch,▁▃▁▃▅▆█
loss,█▁▁▁▁▁▁
accuracy,▁▂▅▇▇██
val_loss,█▅▄▃▃▆▁
val_accuracy,▁▃▆▇▆▄█
_runtime,▁▂▃▅▆▇██████
_timestamp,▁▂▃▅▆▇██████
_step,▁▂▂▃▄▄▅▅▆▇▇█
f1,▁
recall,▁
precision,▁


[INFO] compiling model...
Epoch 1/2
90/90 [==============================] - 178s 2s/step - loss: 28.5772 - accuracy: 0.6645 - val_loss: 0.7813 - val_accuracy: 0.7358
Epoch 2/2
90/90 [==============================] - 175s 2s/step - loss: 1.0915 - accuracy: 0.7277 - val_loss: 0.8748 - val_accuracy: 0.7585
[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/5
90/90 [==============================] - 178s 2s/step - loss: 1.2524 - accuracy: 0.7164 - val_loss: 0.5684 - val_accuracy: 0.8281
Epoch 2/5
90/90 [==============================] - 176s 2s/step - loss: 0.3856 - accuracy: 0.8376 - val_loss: 1.0228 - val_accuracy: 0.6861
Epoch 3/5
90/90 [==============================] - 176s 2s/step - loss: 0.3449 - accuracy: 0.8628 - val_loss: 0.4196 - val_accuracy: 0.8026
Epoch 4/5
90/90 [==============================] - 176s 2s/step - loss: 0.2958 - accuracy: 0.8999 - val_loss: 0.2732 - val_accuracy: 0.8423
Epoch 5/5
90/90 [==============================] - 175s 2s/step - loss: 0.28

epoch,4
loss,0.26472
accuracy,0.91233
val_loss,0.39893
val_accuracy,0.91193
_runtime,1253
_timestamp,1619572804
_step,11
best_val_loss,0.27315
best_epoch,3
f1,0.89796


epoch,▁▃▁▃▅▆█
loss,█▁▁▁▁▁▁
accuracy,▁▃▃▆▇▇█
val_loss,▆▇▄█▂▁▂
val_accuracy,▃▃▅▁▅▆█
_runtime,▁▂▃▄▆▇██████
_timestamp,▁▂▃▄▆▇██████
_step,▁▂▂▃▄▄▅▅▆▇▇█
f1,▁
recall,▁
precision,▁


In [ ]:
modelo = training_model(testes[4])
wandb.run.finish()

wandb: Currently logged in as: thiagosouto (use `wandb login --relogin` to force relogin)


17227776/17225924 [==============================] - 0s 0us/step
[INFO] compiling model...
Epoch 1/2
45/45 [==============================] - 151s 3s/step - loss: 30.6873 - accuracy: 0.6534 - val_loss: 0.4091 - val_accuracy: 0.7922
Epoch 2/2
45/45 [==============================] - 133s 3s/step - loss: 1.7987 - accuracy: 0.7156 - val_loss: 0.3974 - val_accuracy: 0.8141
[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/5
45/45 [==============================] - 136s 3s/step - loss: 0.6806 - accuracy: 0.7083 - val_loss: 0.4297 - val_accuracy: 0.8156
Epoch 2/5
45/45 [==============================] - 133s 3s/step - loss: 0.4863 - accuracy: 0.7647 - val_loss: 0.4329 - val_accuracy: 0.8141
Epoch 3/5
45/45 [==============================] - 132s 3s/step - loss: 0.4648 - accuracy: 0.7632 - val_loss: 0.4109 - val_accuracy: 0.8375
Epoch 4/5
45/45 [==============================] - 133s 3s/step - loss: 0.4452 - accuracy: 0.7838 - val_loss: 0.3718 - val_accuracy: 0.8516
Epoch 5/5
4

epoch,4
loss,0.42052
accuracy,0.79601
val_loss,0.38134
val_accuracy,0.84688
_runtime,974
_timestamp,1619550879
_step,11
best_val_loss,0.37183
best_epoch,3
f1,0.86202


epoch,▁▃▁▃▅▆█
loss,█▂▁▁▁▁▁
accuracy,▂▂▁▅▆▇█
val_loss,▅▄██▅▁▂
val_accuracy,▁▄▄▄▆█▇
_runtime,▁▂▃▄▆▇██████
_timestamp,▁▂▃▄▆▇██████
_step,▁▂▂▃▄▄▅▅▆▇▇█
f1,▁
recall,▁
precision,▁


In [ ]:

iap = ImageToArrayPreprocessor()
print("[INFO] evaluating network...")
testGen = HDF5DatasetGenerator(TEST_HDF5, 30, preprocessors=[iap], classes= args["num_classes"])
classNames = ["COVID", "Normal"]
predictions = modelo.predict(testGen.generator(1) ,verbose=1)
testY = np.array(testGen.db["labels"])
pred = predictions.argmax(axis=1)
print(testY.shape)
print(pred.shape)

!cp -r raiox/vgg16_v2_xrays.model /content/drive/MyDrive/Projeto3/modelos

[INFO] evaluating network...
24/24 [==============================] - 8s 327ms/step
(720,)
(720,)


In [ ]:
print(predictions.shape)

(720, 2)
